# Gravity and Magnetic 3D Inversion

<img align="right" width="300px" src="./images/inversion_grav_mag_thumbnail.png">

This application provides an interface to the open-source [SimPEG](https://simpeg.xyz/) package for the inversion of gravity and magnetic data (fields and tensor components). The application handles:

 - Gravity inversion for the recovery of residual density (g/cc).
 - Scalar magnetic inversion (MAG) for the recovery of susceptibility (SI).
 - Vector magnetic inversion (MVI) for the recovery of 3-component magnetization (effective susceptibility and angles).

All inversions are performed on a 3D octree mesh.

New user? Visit the [**Getting Started**](https://geoapps.readthedocs.io/en/latest/content/installation.html) page.

## Application
The following sections provide details on the different parameters controlling the application. Interactive widgets shown below are for demonstration purposes only.

In [1]:
from geoapps.inversion.potential_fields.application import InversionApp

app = InversionApp(geoh5=r"../../../assets/FlinFlon.geoh5")
app()

## Project Selection

Select and connect to an existing **geoh5** project file containing data

OR 

Select a `*.ui.json` input file to re-load parameters from. See the [Input ui.json](#Input-ui.json) section for details.

In [2]:
app.project_panel

See the [Project Panel](base_application.ipynb#Project-Panel) page for more details.

## Survey Selection

### Object

List of objects available containing geophysical data to be inverted. Currently supports objects of type `Points`, `Curve`, `Surface` and `Grid2D`.

In [3]:
app.objects

Dropdown(description='Object:', index=18, options=(['', None], ['ISO/Iso_Iteration_7_model_5_00e-03', UUID('3f…

### Inversion Type

List of available inversion types. 

- **gravity**: Invert bouguer corrected (mGal) or tensor (Etvos) data for the recovery of residual density (g/cc).
- **magnetic scalar**: Invert magnetic data for the recovery of scalar susceptibility (SI).
- **magnetic vector**: Invert magnetic data for the recovery of 3-component magnetization (effective susceptibility + angles).

In [4]:
app.survey_type_panel

#### Inducing field parameters

For magnetic data only, set the inducing field parameters:

## Data Channel Options

Define the geophysical data to be inverted.

### Channel Options

Set parameters to invividual channels.

In [5]:
app.data_channel_choices.data_channel_options['tmi']

#### Active

Checked if the channel is to be used in the inversion

#### Channel:

Association between the "data channel" and one of the expected "system channel".

#### Uncertainty floor

Estimated data uncertainty floor (constant) value associated with the data channel. 

#### Uncertainty channel

Optionally, use a data channel to assign point-based uncertainties. Uncertainty floor value is ignored if a channel of uncertainties is selected.

## Window Selection

Manual selection of an area of interest and data resolution.

In [6]:
app.window_selection

See the [Map View Selection](view_selection.ipynb#Map-View-Selection) page for more details.

## Spatial information

### Topography

Set the air/ground interface of the inversion model.

In [7]:
app.topography_group()

#### Object

Set the topography based on an object stored in the target `geoh5` project. The Z value of the cells/vertices can be assigned based on a chosen `Data` field.

#### Relative to Sensor

Topography is defined by a fixed vertical offset from a selected object position (vertices or centroids), also referred to as a "draped height".

#### Constant

Topography is defined as a flat surface with constant elevation.

### Sensor Location

Defines the sensors position in 3D space. The vertical position can be referenced to:
- **sensor location**: Use the target [object](#Object) vertices or centroids z-locations
- **topo + radar**: Interpolate the topography to set the elevation. Typically used for gridded
    data with constant draped height or for airborne survey with inaccurate GPS
    elevation (see [radar channel](#Radar-channel-(optional))).

In [8]:
app.sensor()

#### Constant offsets (dx, dy, dz)

Sensor position shifted by a constant offset from the vertices of the selected
object. Typically used for towed systems where the GPS receiver is on the
aircraft.

- dx: East(+) or West(-) offset.
- dy: North(+) or South(-) offset
- dz: Up(+) or Down(-) offset

#### Radar channel (optional)

Channel defining the ground clearance (radar) of each data point.

## Inversion Options

List of parameters controlling the inversion.

In [9]:
app.option_choices

Dropdown(options=('starting model', 'mesh', 'reference model', 'regularization', 'upper-lower bounds', 'detren…

### Starting Model

Initial model used to begin the inversion.


In [10]:
app._starting_model_group()

#### Model

Model object and values selected from any `Points`, `Curve`, `Surface`, `BlockModel` or `Octree` object.
Values are interpolated onto the inversion mesh using a nearest neighbor approach.

#### Constant

Constant background half-space value.

## Mesh

### Option 1: Object
Select an existing `Octree` mesh object from the target geoh5.

In [11]:
app._mesh_octree.objects

Dropdown(description='Object:', options=(['', None], ['ISO/Iso_Iteration_7_model_5_00e-03', UUID('3fde53cf-ea6…

### Option 2: Create

Set parameters used to build of an `Octree` mesh.

In [12]:
app._mesh_octree._parameters

#### Core cell size (u, v, z)

Smallest (core) cell dimensions along 

- u: East-West
- v: North-South
- z: Vertical

As a general rule of thumb, the cell size should be roughly equal or smaller to the data resolution.


####  # Cells below topography


Number of layers of cells at each octree level below the topography surface.
As topography generally extends further, it is common practice to use only
coarse cells once outside of the survey area.


####  # Cells below sensors

Number of layers of cells at each octree level below the observation points.


#### Maximum distance


Parameter controlling the Delaunay triangulation used for the refinement of
the mesh around topography and observation points. Large triangles generated
between points far apart are filtered out, reducing the number of small cells
in regions without data.

####  Horizontal padding

Additional padding distance (m) around the survey extent. Distances are rounded up to the next power of 2.

####  Vertical padding

Additional padding distance (m) added above and below the survey vertical relief. Distances rounded up to the next power of 2.

####  Minimum depth


Minimum thickness of the mesh added at depth below topography.


See the [Octree Mesh Creation](create_octree.ipynb#Mesh-Parameters) app for more details


 <img align="center" width="50%" src="./images/Octree_refinement.png">

## Regularization Panel

Parameters controlling the regularization function.

In [13]:
app.inversion_options['regularization']

### Reference model

Reference model values used to constrain the inversion.

In [14]:
app._reference_model_group()

#### None

No reference model used. This is equivalent as setting the [Alphas](#Alpha-Parameters) `s` parameter to zero.

#### Model

Reference model selected from any `Points`, `Curve`, `Surface`, `BlockModel` or `Octree` object.
Values are interpolated onto the inversion mesh using a nearest neighbor approach.

#### Constant

Constant reference half-space value.

### Scaling (alpha) parameters

In [15]:
app._alphas

Scaling between the components of the regularization function.

See the [SimPEG.API](https://docs.simpeg.xyz/content/api_core/api_Regularization.html#SimPEG.regularization.Simple) for technical details.

### $l_p$-norms

Approximated norms applied to the components of the regularization.

In [16]:
app._norms

See notes on [Sparse and Blocky Norms](https://giftoolscookbook.readthedocs.io/en/latest/content/fundamentals/Norms.html#sparse-and-blocky-norms) for technical details.

### Upper-Lower Bounds

Upper and lower bound constraints applied on physical property model.

In [17]:
app.bound_panel

Bounds can be defined as either:
- Model: Values defined on a cell-by-cell basis. The values are projected onto the inversion mesh using a nearest neighour interpolation. 
- Constant: Constant value defining the bounds.
- None: The inversion uses [$-\infty$, $\infty$] as physical property bounds.

### Detrend (optional)

Remove a polynomial trend from the selected data channels.

In [18]:
app.workspace


In [19]:
import os
app.write_trigger(None)
app.params.write_input_file(name="VectorInversion.ui.json", path='./')
app.detrend_type.value = "all"

In [20]:
app._detrend_panel

#### Order

Order [0, 1 or 2] of the polynomial defining the trend. The 0th order trend corresponds to removing the mean value.

#### Method

- all: Use all data points to compute the trend
- perimeter: Use only the edge points to compute the trend.

### Ignore Values

In [21]:
app.ignore_values

Text(value='', description="Value (i.e. '<0' for no negatives)", style=DescriptionStyle(description_width='ini…

Flag value ignored by the inversion by assigning $\infty$ uncertainties on the data points.

### Optimization

Parameters controlling various aspects of the projected Gauss-Newton inversion algorithm.

In [22]:
app.optimization

#### Max beta Iterations

Maximum number of $\beta$-iterations allowed.
Note that when applying sparse norms, the inversion may require $>20$ iterations to converge.


#### Target misfit

Target data misfit where $\chi=1$ corresponds to $\phi_d=N$ (number of data). (See documentation on [Data Misfit and Uncertainties](https://giftoolscookbook.readthedocs.io/en/latest/content/fundamentals/Uncertainties.html#data-misfit-and-uncertainties) for mathematical details)


#### Starting trade-off ($\beta$)

**ratio**: Factor multiplying the initial $\beta$ defined by the ratio between the initial misfit and regularization:

\begin{equation}
\beta_0 = \gamma * \frac{\phi_d}{ \phi_m}
\end{equation}

**value**: Fixed $\beta$ value specified by the user.

#### Max CG Iterations

Maximum number of Conjugate Gradient (CG) iterations per Gauss-Newton solve.


#### CG Tolerance

Threshold on the minimum Conjugate Gradient (CG) step to end the Gauss-Newton solve.


#### Max CPUs

Maximum number of threads used for the parallelization. Defaults to half the system thread count.


#### Number of tiles

Number of data tiles used by the mesh decoupling algorithm. Nested sensitivities for each data blocks are interpolated to the global inversion mesh with a volumetric weighted averaging scheme.

## Output panel

Setup the inversion and launch the process.

### Output Name

In [23]:
app.ga_group_name

Text(value='VectorInversion', description='Save as:', style=DescriptionStyle(description_width='initial'))

Name given to the inversion group added to the Geoscience ANALYST project.

### Working directory

In [24]:
app.export_directory

FileChooser(path='C:\Users\dominiquef\Documents\GIT\mira\geoapps\docs\content\applications', filename='', titl…

Specify the working directory where the inversion occurs. 

**It is highly recommended to chose a location on a solid-state drive (SSD) to fully take advantage of the `Dask` parallelization.**

### Write input

In [25]:
app.write

Button(description='Write input', icon='check', style=ButtonStyle())


Click to write the input parameters to a `*.json` file and a workspace (.geoh5) with all required entities. 

### Run Inversion: 

In [26]:
app.trigger

Button(button_style='success', description='Compute', icon='check', style=ButtonStyle(), tooltip='Run computat…

Launch the inversion routine. Results will be written directly to the target `geoh5` project.

## Input ui.json 

This application relies on a [structured json](./VectorInversion.ui.json) file to store the parameters and run the program.
The `ui.json` can be used to re-load the parameters from a previous run by selecting the `ui.json` file from the [Project Selection](#Project-Selection) widget, instead of a `geoh5` file. 


### Command line execution

The same input `ui.json` file can be used to run the program from command line:

```
activate geoapps
python -m geoapps.inversion.driver [Name].ui.json
```

This assumes that the geoapps have been installed and that the reference `*.ui.json` and `*.geoh5` project are accessible in the current directory.

### Geoscience ANALYST Pro v3.4

Geoscience ANALYST Pro users (v3.4) can execute this application directly from an active session with a drag & drop of a `*.ui.json` file to the `Viewport`. 

![GAPro_Octree](images/GA_pro_grav_mag_inversion.gif "ga_pro")

In [27]:
import matplotlib.pyplot as plt
app.figure.savefig("images/inversion_thumbnail.png", dpi=300, bbox_inches="tight")